In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
import random
import math

In [ ]:
wandb.login()
# 7adbc9679e82154709a6abc3ba6faeb396927426

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jpca92 (grupo_8) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.login()

True

In [ ]:
import os
num_workers = os.cpu_count()
print(num_workers)

12


In [ ]:
# Download and install the code
import sys

!git clone https://github.com/Alexandre-Delplanque/HerdNet
!cd '/content/HerdNet' && python setup.py install

sys.path.append('/content/HerdNet')

Cloning into 'HerdNet'...
remote: Enumerating objects: 570, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 570 (delta 149), reused 132 (delta 132), pack-reused 377 (from 2)
Receiving objects: 100% (570/570), 205.52 KiB | 17.13 MiB/s, done.
Resolving deltas: 100% (370/370), done.
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_d

# Montar el google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/Colab Notebooks/trabajo_grado/data.zip' /content/
!unzip -oq /content/data.zip -d /content

Mounted at /content/drive


In [ ]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

# Training, validation and test datasets

In [ ]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import CSVDataset, FolderDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT
# from animaloc.datasets.folder import FolderDataset
patch_size = 512
num_classes = 7
down_ratio = 2

train_dataset = FolderDataset(
    csv_file = '/content/data/train_fullset_patches/gt.csv',
    root_dir = '/content/data/train_fullset_patches',
    albu_transforms = [
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
        A.Blur(blur_limit=15, p=0.2),
        A.Normalize(p=1.0)
        ],
    end_transforms = [MultiTransformsWrapper([
        FIDT(num_classes=num_classes, down_ratio=down_ratio),
        PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
        ])]
    )

val_dataset = CSVDataset(
    csv_file = '/content/data/val_patches/gt.csv',
    root_dir = '/content/data/val_patches',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(
    csv_file = '/content/data/test.csv',
    root_dir = '/content/data/test',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

# Dataloaders

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,  # o el que aguante la GPU
    shuffle=True,
    num_workers= num_workers,
    pin_memory=True
)

In [ ]:
val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

In [ ]:
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

# Define HerdNet for training

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

Downloading: "http://dl.yf.io/dla/models/imagenet/dla34-ba72cf86.pth" to /root/.cache/torch/hub/checkpoints/dla34-ba72cf86.pth
100%|██████████| 60.3M/60.3M [00:04<00:00, 15.2MB/s]


# Create the Trainer

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 10

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=work_dir,
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

In [ ]:
wandb.init(
    project="herdnet_v2",
    name=f"JP_train_lr={lr}_wd={weight_decay}_ep={epochs}",
    config={
        "learning_rate": lr,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "model": "HerdNet",
        "train_dataset": "/content/data/train_fullset_patches/gt.csv",
        "val_dataset": "/content/data/val_patches/gt.csv"
    }
)



In [ ]:
trainer.start(warmup_iters=100,
              checkpoints='best',
              select='max',
              validate_on='f1_score',
              wandb_flag=True)

INFO:20250505_training:[TRAINING] - Epoch: [1] [   1/2616] eta: 8:07:14 lr: 0.000002 loss: 9294.2119 (9294.2119) focal_loss: 9292.1289 (9292.1289) ce_loss: 2.0831 (2.0831) time: 11.1753 data: 4.3383 max mem: 32493


[TRAINING] - Epoch: [1] [   1/2616] eta: 8:07:14 lr: 0.000002 loss: 9294.2119 (9294.2119) focal_loss: 9292.1289 (9292.1289) ce_loss: 2.0831 (2.0831) time: 11.1753 data: 4.3383 max mem: 32493


INFO:20250505_training:[TRAINING] - Epoch: [1] [  51/2616] eta: 0:43:18 lr: 0.000051 loss: 5939.7261 (7560.9678) focal_loss: 5939.2539 (7559.7534) ce_loss: 0.5177 (1.2144) time: 0.7958 data: 0.0003 max mem: 32637


[TRAINING] - Epoch: [1] [  51/2616] eta: 0:43:18 lr: 0.000051 loss: 5939.7261 (7560.9678) focal_loss: 5939.2539 (7559.7534) ce_loss: 0.5177 (1.2144) time: 0.7958 data: 0.0003 max mem: 32637


INFO:20250505_training:[TRAINING] - Epoch: [1] [ 101/2616] eta: 0:37:59 lr: 0.000100 loss: 1118.6747 (4932.2346) focal_loss: 1118.5695 (4931.4621) ce_loss: 0.1845 (0.7725) time: 0.7975 data: 0.0003 max mem: 32637


...

[VALIDATION] - Epoch: [10] [521/532] eta: 0:00:00 n: 1 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 0.0716 data: 0.0073 max mem: 34138


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

[VALIDATION] - Epoch: [10] [531/532] eta: 0:00:00 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 0.0744 data: 0.0077 max mem: 34138
[VALIDATION] - Epoch: [10] [532/532] eta: 0:00:00 n: 2 recall: 1.0 precision: 0.29 f1-score: 0.44 MAE: 5.0 MSE: 25.0 RMSE: 5.0 time: 0.0743 data: 0.0076 max mem: 34138
[VALIDATION] - Epoch: [10] Total time: 0:00:39 (0.0745 s / it)
[VALIDATION] - Epoch: [10] f1_score: 0.6549


ce_loss,▂▄█▅▅▁▂▁▂▂▂▃▁▁▂▅▄▁▂▂▂▇▁▂▁▁▄▂▁▁▁▂▁▂▂▁▂▂▁▁
epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
f1_score,▁▄▆▇▇█▇▇█▇
focal_loss,█▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▁▁▁▁▁▁▁▁▁
best_validation,0.70917
ce_loss,0.05566
epoch,10
f1_score,0.65487
focal_loss,0.04662


LossWrapper(
  (model): HerdNet(
    (base_0): DLA(
      (base_layer): Sequential(
        (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level0): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level1): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level2): Tree(
        (tree1): BasicBlock(
          (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-0

In [ ]:
wandb.finish()

# Test del modelo recien entrenado

In [ ]:
# -----------------------------
# Test the model entrenado
# -----------------------------

import os
import torch
from animaloc.models import HerdNet
from animaloc.models import load_model
from animaloc.eval import HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

# Ruta al modelo entrenado con metadatos
pth_path = '/content/output/best_model.pth'

# Crear carpeta de salida
test_dir = '/content/test_output'
os.makedirs(test_dir, exist_ok=True)

# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path=pth_path)

In [ ]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [ ]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
INFO:20250505_evaluation:test [  1/258] eta: 0:15:36 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 3.6282 data: 0.6958 max mem: 34138


test [  1/258] eta: 0:15:36 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 3.6282 data: 0.6958 max mem: 34138


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [ 11/258] eta: 0:14:44 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 3.5654 data: 0.6379 max mem: 34138


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

...

test [241/258] eta: 0:01:01 n: 13 recall: 0.23 precision: 0.43 f1-score: 0.3 MAE: 6.0 MSE: 36.0 RMSE: 6.0 time: 3.1159 data: 0.5348 max mem: 34138


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [251/258] eta: 0:00:27 n: 2 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 8.0 MSE: 64.0 RMSE: 8.0 time: 3.1080 data: 0.5277 max mem: 34138


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [258/258] eta: 0:00:03 n: 3 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 3.0 MSE: 9.0 RMSE: 3.0 time: 3.0928 data: 0.5294 max mem: 34138
test Total time: 0:14:38 (3.4048 s / it)


In [ ]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

F1 score = 59%


In [ ]:
# Guardar resultados
detections = test_evaluator.results
detections

,class,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,675,0.620741,0.733800,0.672552,0.316476,3.478261,49.246377,7.017576,0.532880
1,2,349,0.848138,0.216533,0.344988,0.077882,10.560000,252.080000,15.877027,0.422211
2,3,477,0.905660,0.752613,0.822074,0.086681,1.314050,5.099174,2.258135,0.827422
3,4,74,0.297297,0.090535,0.138801,0.000000,3.182927,24.695122,4.969419,0.136867
4,5,36,0.805556,0.087087,0.157182,0.093750,4.152778,46.763889,6.838413,0.284817
5,6,688,0.340116,0.207447,0.257709,0.486842,8.520000,196.680000,14.024265,0.155219
6,binary,2299,0.833841,0.454696,0.588488,0.000000,8.306202,256.019380,16.000606,0.336430
